In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from hyperopt import hp, tpe, fmin, Trials

# Load data
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define parameter grid for GridSearchCV
param_grid_gridsearch = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# Define parameter grid for RandomizedSearchCV
param_dist_randomsearch = {
    'max_depth': [3, 5, 7, 9, 11, 13, 15],
    'learning_rate': np.linspace(0.001, 0.1, 100),
    'n_estimators': np.arange(100, 1000, 100)
}


# Define objective function for Hyperopt
def objective(params):
    model = XGBClassifier(**params, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return -accuracy  # Minimize negative accuracy

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=XGBClassifier(random_state=42),
                           param_grid=param_grid_gridsearch,
                           scoring='accuracy',
                           cv=5,
                           verbose=1)
grid_search.fit(X_train, y_train)
print("Grid Search Best Parameters:", grid_search.best_params_)
best_grid_model = grid_search.best_estimator_

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=XGBClassifier(random_state=42),
                                   param_distributions=param_dist_randomsearch,
                                   n_iter=100,
                                   scoring='accuracy',
                                   cv=5,
                                   verbose=1,
                                   random_state=42)
random_search.fit(X_train, y_train)
print("Random Search Best Parameters:", random_search.best_params_)
best_random_model = random_search.best_estimator_


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Grid Search Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 200}
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Random Search Best Parameters: {'n_estimators': 700, 'max_depth': 3, 'learning_rate': 0.003}
  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'

In [2]:
# Define search space for Hyperopt
param_space_hyperopt = {
    'max_depth': hp.choice('max_depth', np.arange(3, 16, dtype=int)),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'n_estimators': hp.choice('n_estimators', np.arange(100, 1000, 100, dtype=int))
}

# Perform Hyperopt
trials = Trials()
best_hyperopt = fmin(fn=objective,
                     space=param_space_hyperopt,
                     algo=tpe.suggest,
                     max_evals=100,
                     trials=trials,
                     rstate=np.random.default_rng(42),
                     return_argmin=False
                     )
print("Hyperopt Best Parameters:", best_hyperopt)
best_hyperopt_model = XGBClassifier(**best_hyperopt, random_state=42)
best_hyperopt_model.fit(X_train, y_train)

# Evaluate best models
y_pred_grid = best_grid_model.predict(X_test)
accuracy_grid = accuracy_score(y_test, y_pred_grid)
print("Accuracy of Grid Search Best Model:", accuracy_grid)

y_pred_random = best_random_model.predict(X_test)
accuracy_random = accuracy_score(y_test, y_pred_random)
print("Accuracy of Random Search Best Model:", accuracy_random)

y_pred_hyperopt = best_hyperopt_model.predict(X_test)
accuracy_hyperopt = accuracy_score(y_test, y_pred_hyperopt)
print("Accuracy of Hyperopt Best Model:", accuracy_hyperopt)

100%|██████████| 100/100 [00:29<00:00,  3.35trial/s, best loss: -1.0]
Hyperopt Best Parameters: {'learning_rate': 0.017387534125014405, 'max_depth': 13, 'n_estimators': 100}
Accuracy of Grid Search Best Model: 1.0
Accuracy of Random Search Best Model: 1.0
Accuracy of Hyperopt Best Model: 1.0
